El siguiente paso es extraer información detallada de los 10 principales actores de cada una de las películas extraidas en la fase 1 utilizando Selenium (desde la página e IMDB). Debereis solo tener en cuenta los 10 primeros actores de cada película o corto (en caso de que los haya)La información que deberás extraer es:

Nombre.

Año nacimiento.

¿Por qué es conocido?

¿Qué hace?

Premios.

NOTA: La información de la API deberá ser almacenada en una lista de tuplas. Cada tupla corresponderá a una película. Siguiendo el siguiente ejemplo:

In [ ]:
[("Adam Sadler", 1966, ["El aguador", "Little Nicky", "Un papá genial", "El chico ideal" ], ["Reparto", "Producción", "Guion"] , ["American Comedy Awards, USA", "Annie Awards"],   ),  
...]